# Topic Modelling Corpus for Habrahabr

In [10]:
import os
import json

In [75]:
DATASET_PATH = './dataset_full/habr_posts/'
post_ids = sorted(int(filename) for filename in os.listdir(DATASET_PATH) if not filename.startswith('.'))
def load_post(post_id):
    with open(os.path.join(DATASET_PATH, str(post_id))) as fin:
        post = json.load(fin)
    return post

## Модальности

 1. `words`
 2. `author`
 3. `users`
 4. `tags`
 5. `hubs` (hub/apps_design)

In [100]:
import nltk
import urlparse
import collections
import BeautifulSoup

In [ ]:
import nltk
import html2text
html2text.config.INLINE_LINKS = False
html2text.config.SKIP_INTERNAL_LINKS = True
html2text.config.IGNORE_ANCHORS = True
html2text.config.IGNORE_EMPHASIS = True
html2text.config.IGNORE_IMAGES = True
html2text.config.BYPASS_TABLES = False

In [33]:
import html2text
content_text = html2text.html2text(post['content_html'])
tokens = nltk.tokenize.wordpunct_tokenize(content_text)

In [25]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [104]:
%%time

def html_to_plain(html):
    text = html2text.html2text(post['content_html'])

def post_to_corpus_line(post, morph):
    post_id = post['_id']
    author = post['author']
    tags = post['tags']

    # 1. words
    words = collections.Counter()
    
    soup = BeautifulSoup.BeautifulSoup(post['content_html'])
    text_parts = soup.findAll(text=True)
    content_text = ''.join(text_parts)
    space_chars = u'«»“”’*…/_.\\'
    for c in space_chars:
        content_text = content_text.replace(c, ' ')
    tokens = nltk.tokenize.wordpunct_tokenize(content_text)
    tokens = nltk.word_tokenize(content_text)
    for token in tokens:
        if len(token) > 2:
            token = token.lower().replace(u'ё', u'е')
            word = morph.parse(token)[0].normal_form
            if len(word) > 0:
                words[word] += 1
    
    # 2. users
    users = collections.Counter()
    
    def pass_comments(comments, users):
        for comment in comments:
            if not comment['banned'] and comment.get('username') is not None:
                username = comment['username']
                users[username] += 1
            pass_comments(comment['replies'], users)
    pass_comments(post['comments'], users)

    # 3. hubs
    
    def parse_hub_id(hub_pair):
        url_parts = filter(lambda s: len(s) > 0, urlparse.urlsplit(hub_pair[1]).path.split('/'))
        if len(url_parts) >= 2:
            hub_id = '_'.join(url_parts[-2:])
            return hub_id
        
    hubs = []
    for hub_pair in post['hubs']:
        hub_id = parse_hub_id(hub_pair)
        if hub_id:
            hubs.append(hub_id)
    
    # construct bag-of-words
    
    def construct_bow(words):
        return [
            (
                word.replace(' ', '_').replace(':', '_').replace('|', '_').replace('\t', '_') + 
                ('' if cnt == 1 else ':%g' % cnt)
            )
            for word, cnt in words.iteritems()
        ]
    
    parts = (
        ['\'%d' % post_id] + 
        ['|words'] + construct_bow(words) +
        ['|author'] + construct_bow({author: 1} if author is not None else {}) +
        ['|users'] + construct_bow(users) + 
        ['|tags'] + construct_bow({tag: 1 for tag in tags}) +
        ['|hubs'] + construct_bow({hub_id: 1 for hub_id in hubs})
    )
    return ' '.join(parts)
    

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 89.9 µs


In [105]:
print post_to_corpus_line(load_post(251015), morph)

'251015 |words требоваться комментарий обработка:4 чуть полнота:2 немного:2 охрана:2 список нормальный житель:2 постановка выдержка где-нибудь получать:2 раздать vpn распространить записать пол страна:4 конец:2 нереальный далёкий:4 распознать:3 хоть последний под:5 нежелательно пара искать тестер копейка контроль люба стратегический защита:2 рушить российский сырьё единица:3 человек:11 система:4 точно разработчик более научиться образцов фельдъегерский общий категория объём результат:3 экономить пример:2 задавать случай croc существенно:2 дойти:3 нерв сложный знание:2 коллега массив итоговый ввод много:3 учитывать качество проверить:3 несколько:2 мыслить обводить парсить агроном рукописный:2 стоить:2 распознавание-проверка процент:2 двухтысячный там:4 нагрузка мало механизм:2 живой подтвердить профессионал озвучить следующий:3 случайный роскошь что:15 начать:4 встречаться слой попросить чаща:2 тут:2 достаточно готовый:2 отчёт:5 успешно следить:2 миллион собранный боком подавать условие

In [ ]:
with open('habrahabr_corpus.txt', 'w') as fout:
    for post_id in post_ids:
        line = post_to_corpus_line(load_post(post_id), morph)
        fout.write(line.encode('utf8') + '\n')